# Fig 2

Fig2(ノイズ付きのHeisenberg Modelのシミュレーション)を再現するためのコードを示す．
計算量が多いので，ノイズ付きシミュレーションを一回行うためののコードが書いてある．

In [2]:
import numpy as np
from openfermion.linalg import get_sparse_operator
from openfermion import QubitOperator
from qulacs import Observable, QuantumState
from qulacs.gate import *
from qulacs.state import inner_product
from scipy.linalg import expm

In [3]:
def Derive_q(S:int,s:float):
    """
    solve Simultaneous equations
    returns vector q as numpy.array
    """
    s = (S-1)/2
    M = []
    D = [0]*S
    D[1] = 1
    for i in range(S):
        M.append([])
        for j in range(S):
            M[-1].append((-s+j)**(i))
    M = np.array(M)
    q = np.linalg.inv(M)@D
    return q

def HeisenbergQubitOperator(J,h,N):
    """
    returns hamiotonian as openfermion.ops.operators.qubit_operator.QubitOperator
    """
    hamiltonian = QubitOperator()
    spin2_list = []
    spin1_list = []

    for i in range(N):
        spin1_list.append("Z{}".format(i))
    for i in range(N-1):
        for j in ["X","Y","Z"]:
            spin2_list.append("{}{} {}{}".format(j,i,j,i+1))
    for i in range(N):
        hamiltonian += -h*QubitOperator(spin1_list[i])
    for i in range(3*(N-1)):
        hamiltonian += -J*QubitOperator(spin2_list[i])
    return hamiltonian

#print(type(HeisenbergQubitOperator(J,h,N)))
    

def Heisenberg_Hamiltonian(J,h,N):
    """
    returns Heisenberg hamiltonian as array
    """
    hamiltonian = HeisenbergQubitOperator(J,h,N)
    type(get_sparse_operator(hamiltonian,n_qubits=N))
    hamiltonian = get_sparse_operator(hamiltonian).toarray()
    return hamiltonian

#print(type(get_sparse_operator(HeisenbergQubitOperator(J,h,N),n_qubits=N)))

def Propagator(hamiltonian,t:float):
    """
    inputs: hamiltonian as `np.array`, t:float
            construct a time evolution of Heisenberg Hamiltonian: U=e^{-iHt}
    returns: array of the unitary matrix
    """
    unitary_mtrx = expm(-1j * hamiltonian * t)
    return unitary_mtrx

def H_Diff(S:int,s:float,delta:float,hamiltonian):
    """
    inputs: hamiltonian as `np.array`, and some `int` and `float`s.
    construct HOA hamilotonian by size S and distance of neighboring points delta
    returns array of the HOA hamilotonian
    """
    veq_q = Derive_q(S=S,s=s)
    #s=(S-1)/2
    H = Propagator(hamiltonian,-s*delta)*veq_q[0]
    for i in range(1,S):
        H+=Propagator(hamiltonian,(-s+i)*delta)*veq_q[i]
    return 1j/delta * H


In [4]:
from scipy import stats
from collections import Counter
import numpy as np

J,h,N = 1,1,6
s_max = 9
delta = 0.01
s = (s_max-1)/2
N_meas = 10**6

def ctrl_gate(U,c):
    return add(P0(c),merge(P1(c),U))

def inv_ctrl_gate(U,c):
    return add(P1(c),merge(P0(c),U))

def after_SWAP_state(N,unitary_gate):
    state = QuantumState(N+2)
    for i in range(N):
        H(i).update_quantum_state(state)
    H(N).update_quantum_state(state)
    ctrl_gate(unitary_gate,N).update_quantum_state(state)
    S(N).update_quantum_state(state) ## for imaginary
    H(N).update_quantum_state(state)
    return state

def Generate_bernoulli(prob,N_meas):
    x = stats.bernoulli.rvs(p=prob, size=N_meas)
    return sum(x)

def measure(state,m):
    if m==0:
        P0(0).update_quantum_state(state)
    else:
        P1(0).update_quantum_state(state)
    state.normalize(state.get_squared_norm())
    return state

def Hadamard_test(state,uni_matrix,ancilla:int,N_meas:int):
    circuit = merge(merge(H(ancilla),ctrl_gate(uni_matrix,ancilla)) , H(ancilla))
    circuit.update_quantum_state(state)
    state.normalize(state.get_squared_norm())
    p1 = 1-state.get_zero_probability(ancilla)
    b = Generate_bernoulli(p1,N_meas)
    return 1-2*b/N_meas

hamiltonian = Heisenberg_Hamiltonian(J,h,N)
veq_q = Derive_q(S=s_max,s=s)

#print(hamiltonian)

energy = 0
for i in range(s_max):
    unitary_gate = DenseMatrix([i for i in range(N)],Propagator(hamiltonian,delta*(-s+i)))
    state = after_SWAP_state(N,unitary_gate)
    uni_exp = Hadamard_test(state=state,uni_matrix=Z(N),ancilla=N+1,N_meas=N_meas)
    energy += (1/delta)*veq_q[i]*uni_exp

#Energy Expaectation
print(energy)

-5.016350952380962
